In [1]:
!pip install flask flask-ngrok

In [11]:
!pip install flask pyngrok

In [14]:
from pyngrok import ngrok

!ngrok config add-authtoken 39zAyOFXMR5RdRkwBnhxPluRXRw_784qMT6auEHektZz1mhw6

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [15]:
from flask import Flask, request, jsonify
import sqlite3
from datetime import datetime
from pyngrok import ngrok
import threading

app = Flask(__name__)

# -----------------------
# DATABASE
# -----------------------
def init_db():
    conn = sqlite3.connect("reports.db")
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS reports (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        type TEXT,
        location TEXT,
        severity TEXT,
        description TEXT,
        status TEXT DEFAULT 'Pending',
        timestamp TEXT
    )
    """)
    conn.commit()
    conn.close()

init_db()

# -----------------------
# ROUTES
# -----------------------

@app.route("/")
def home():
    return jsonify({"message": "Backend Running Successfully"})

@app.route("/report", methods=["POST"])
def report():
    data = request.json

    conn = sqlite3.connect("reports.db")
    cursor = conn.cursor()

    cursor.execute("""
    INSERT INTO reports (type, location, severity, description, timestamp)
    VALUES (?, ?, ?, ?, ?)
    """, (
        data["type"],
        data["location"],
        data["severity"],
        data["description"],
        datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ))

    conn.commit()
    conn.close()

    return jsonify({"message": "Report Submitted"})

@app.route("/admin")
def admin():
    conn = sqlite3.connect("reports.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM reports")
    rows = cursor.fetchall()
    conn.close()

    result = []
    for row in rows:
        result.append({
            "id": row[0],
            "type": row[1],
            "location": row[2],
            "severity": row[3],
            "description": row[4],
            "status": row[5],
            "timestamp": row[6]
        })

    return jsonify(result)

@app.route("/update/<int:id>")
def update(id):
    conn = sqlite3.connect("reports.db")
    cursor = conn.cursor()
    cursor.execute("UPDATE reports SET status='Resolved' WHERE id=?", (id,))
    conn.commit()
    conn.close()
    return jsonify({"message": "Status Updated"})


# -----------------------
# RUN SERVER WITH NGROK
# -----------------------

def run():
    app.run(port=5000)

# Start Flask in thread
thread = threading.Thread(target=run)
thread.start()

# Create public tunnel
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Public URL: NgrokTunnel: "https://marybelle-castled-luis.ngrok-free.dev" -> "http://localhost:5000"
